In [1]:
from src.agent import HelpDeskAgent
from src.configs import Settings
from src.tools.search_xyz_manual import search_xyz_manual
from src.tools.search_xyz_qa import (
    search_xyz_qa,
)

settings = Settings()


In [2]:
agent = HelpDeskAgent(
    settings=settings,
    tools=[search_xyz_manual, search_xyz_qa],
)

In [3]:
question = """
お世話になっております。

現在、XYZシステムの利用を検討しており、以下の2点についてご教示いただければと存じます。

1. パスワードに利用可能な文字の制限について
当該システムにてパスワードを設定する際、使用可能な文字の範囲（例：英数字、記号、文字数制限など）について詳しい情報をいただけますでしょうか。安全かつシステムでの認証エラーを防ぐため、具体的な仕様を確認したいと考えております。

2. 最新リリースの取得方法について
最新のアップデート情報をどのように確認・取得できるかについてもお教えいただけますと幸いです。

お忙しいところ恐縮ですが、ご対応のほどよろしくお願い申し上げます。
"""

# question = """
# お世話になっております。

# 現在、XYZシステムを利用を検討しており、以下の点についてご教示いただければと存じます。

# 1. 特定のプロジェクトに対してのみ通知を制限する方法について

# 2. パスワードに利用可能な文字の制限について
# 当該システムにてパスワードを設定する際、使用可能な文字の範囲（例：英数字、記号、文字数制限など）について詳しい情報をいただけますでしょうか。安全かつシステムでの認証エラーを防ぐため、具体的な仕様を確認したいと考えております。

# お忙しいところ恐縮ですが、ご対応のほどよろしくお願い申し上げます。

# """

## 計画ステップ

In [4]:
input_data = {"question": question}

plan_result = agent.create_plan(state=input_data)

2025-10-13 05:10:11,421 INFO 🚀 計画生成処理を開始します...
2025-10-13 05:10:11,422 INFO OpenAIにリクエストを送信中...
2025-10-13 05:10:13,801 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-13 05:10:13,823 INFO ✅ OpenAIからのレスポンスを正常に受信しました
2025-10-13 05:10:13,823 INFO 計画生成が完了しました！


In [6]:
print(type(plan_result))

import pprint
pprint.pprint(plan_result)

<class 'dict'>
{'plan': ['XYZシステムのパスワード設定における使用可能な文字の種類（英数字、記号など）について調べる',
          'XYZシステムのパスワード設定における文字数制限について調べる',
          'XYZシステムの最新リリース情報を確認する方法について調べる',
          'XYZシステムの最新リリースを取得する方法について調べる']}


In [7]:
plan_result["plan"]

['XYZシステムのパスワード設定における使用可能な文字の種類（英数字、記号など）について調べる',
 'XYZシステムのパスワード設定における文字数制限について調べる',
 'XYZシステムの最新リリース情報を確認する方法について調べる',
 'XYZシステムの最新リリースを取得する方法について調べる']

## ツール選択ステップ

In [28]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "is_completed": False,
}

In [29]:
select_tool_result = agent.select_tools(state=input_data)

In [ ]:
select_tool_result

In [31]:
select_tool_result["messages"][-1]

{'role': 'assistant',
 'tool_calls': [{'id': 'call_zM2g5tqao9s01G4uYwCtplrr',
   'function': {'arguments': '{"keywords":"パスワード 設定 文字制限"}',
    'name': 'search_xyz_manual'},
   'type': 'function'}]}

In [ ]:
select_tool_result["messages"]

## ツール実行ステップ

In [33]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": select_tool_result["messages"],
    "is_completed": False,
}

In [ ]:
tool_results = agent.execute_tools(state=input_data)

In [ ]:
tool_results["tool_results"][0][0].results

In [ ]:
tool_results

## サブタスク回答

In [37]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": tool_results["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
}

In [38]:
subtask_answer = agent.create_subtask_answer(state=input_data)

In [ ]:
subtask_answer

In [ ]:
print(subtask_answer["subtask_answer"])

## リフレクション

In [41]:
input_data = {
    "question": question,
    "plan": plan_result["plan"],
    "subtask": plan_result["plan"][0],
    "challenge_count": 0,
    "messages": subtask_answer["messages"],
    "tool_results": tool_results["tool_results"],
    "is_completed": False,
    "subtask_answer": subtask_answer["subtask_answer"],
}

In [42]:
reflection_result = agent.reflect_subtask(state=input_data)

In [ ]:
reflection_result

In [ ]:
# 最初に選択されたツールを確認
print(reflection_result["messages"][2]["tool_calls"][0]["function"]["name"])

In [ ]:
# リフレクション結果の確認
print("is_completed =", reflection_result["reflection_results"][0].is_completed)
print("advice =", reflection_result["reflection_results"][0].advice)